In [70]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import Dense, Dropout, LSTM, Embedding, Reshape, Input, InputLayer, Concatenate
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model

In [71]:
n_samples = 20000
sample_inputs = []
sample_outputs = []
n_tokens = 61

patterns = [
    lambda i: list(range(i, i+3)),
    lambda i: list(range(i, i-3, -1))
]

for n in range(n_samples):
    sample_in = []
    sample_out = []
    for line_number in range(3):
        i = np.random.randint(10,50)
        #line = list(range(i, i+3))
        pattern = np.random.randint(len(patterns))
        line = patterns[pattern](i)
        '''if line_number == 0:
            sample_in.append([line[0]] + line)
        else:
            sample_in.append([0] + line)'''
        
        sample_in.append([line[0]] + line)
            
        sample_out.append(line + [60])
        
        
        #sample.append(np.arange(i, i+3))
    #samples.append(sample)
    sample_inputs.append(sample_in)
    sample_outputs.append(sample_out)
    
X = np_utils.to_categorical(sample_inputs, num_classes=n_tokens)
Y = np_utils.to_categorical(sample_outputs, num_classes=n_tokens)

In [72]:
sample_inputs[:10]

[[[15, 15, 16, 17], [18, 18, 19, 20], [17, 17, 18, 19]],
 [[28, 28, 27, 26], [32, 32, 33, 34], [22, 22, 21, 20]],
 [[48, 48, 47, 46], [38, 38, 39, 40], [39, 39, 38, 37]],
 [[20, 20, 21, 22], [39, 39, 38, 37], [44, 44, 43, 42]],
 [[28, 28, 27, 26], [31, 31, 30, 29], [20, 20, 19, 18]],
 [[34, 34, 33, 32], [27, 27, 28, 29], [24, 24, 25, 26]],
 [[22, 22, 23, 24], [31, 31, 32, 33], [18, 18, 19, 20]],
 [[45, 45, 46, 47], [14, 14, 13, 12], [30, 30, 29, 28]],
 [[39, 39, 38, 37], [25, 25, 26, 27], [41, 41, 42, 43]],
 [[13, 13, 14, 15], [39, 39, 40, 41], [16, 16, 17, 18]]]

In [117]:
latent_dim = 512

# Keras Model

inputs = [Input(shape=(None, n_tokens)) for i in range(3)]
lstms = [LSTM(latent_dim, return_state=True, return_sequences=True) for i in range(3)]

lstm_out, lstm_h, lstm_c = [None, None, None], [None, None, None], [None, None, None]
for i in range(3):
    if i > 0:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i], initial_state=[lstm_h[i-1], lstm_c[i-1]])
    else:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i])
        
denses = [Dense(n_tokens, activation='softmax') for i in range(3)]
outputs = [denses[i](lstm_out[i]) for i in range(3)]

model = Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary(line_length=200)

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
input_36 (InputLayer)                                             (None, None, 61)                            0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
lstm_24 (LSTM)                                                    [(None, None, 512), (None, 512), (None, 512 1175552                 input_36[0][0]                                                

In [118]:
model.fit([X[:,0,:], X[:,1,:], X[:,2,:]], [Y[:,0,:], Y[:,1,:], Y[:,2,:]], batch_size=64, epochs=20)

Epoch 1/20
20000/20000 [==============================] - 40s 2ms/step - loss: 5.5128 - dense_23_loss: 2.0342 - dense_24_loss: 1.7703 - dense_25_loss: 1.7083
Epoch 2/20
20000/20000 [==============================] - 36s 2ms/step - loss: 1.1491 - dense_23_loss: 0.5398 - dense_24_loss: 0.3285 - dense_25_loss: 0.2807
Epoch 3/20
20000/20000 [==============================] - 36s 2ms/step - loss: 0.6769 - dense_23_loss: 0.2688 - dense_24_loss: 0.2071 - dense_25_loss: 0.2011
Epoch 4/20
20000/20000 [==============================] - 37s 2ms/step - loss: 0.6076 - dense_23_loss: 0.2240 - dense_24_loss: 0.1940 - dense_25_loss: 0.1896
Epoch 5/20
20000/20000 [==============================] - 35s 2ms/step - loss: 0.5822 - dense_23_loss: 0.2110 - dense_24_loss: 0.1879 - dense_25_loss: 0.1832
Epoch 6/20
20000/20000 [==============================] - 36s 2ms/step - loss: 0.5682 - dense_23_loss: 0.2059 - dense_24_loss: 0.1838 - dense_25_loss: 0.1785
Epoch 7/20
20000/20000 [============================

In [106]:
generator_inputs = [Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens))]
generator_lstms = [LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True)]
generator_lstm_out, generator_lstm_h, generator_lstm_c = [None, None, None], [None, None, None], [None, None, None]
generator_lstm_out[0], generator_lstm_h[0], generator_lstm_c[0] = generator_lstms[0](
    generator_inputs[0]
)
generator_denses = [Dense(n_tokens, activation='softmax')]
generator_outputs = [generator_denses[0](generator_lstm_out[0])]

In [107]:
generator_model_line0 = Model(generator_inputs[0], [generator_outputs[0], generator_lstm_h[0], generator_lstm_c[0]])
generator_model_line0.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        (1, None, 61)             0         
_________________________________________________________________
lstm_22 (LSTM)               [(1, None, 256), (1, 256) 325632    
_________________________________________________________________
dense_21 (Dense)             (1, None, 61)             15677     
Total params: 341,309
Trainable params: 341,309
Non-trainable params: 0
_________________________________________________________________


In [108]:
generator_lstms[0].set_weights(lstms[0].get_weights())
generator_denses[0].set_weights(denses[0].get_weights())

In [115]:
generator_model_line0.reset_states()
line = [14]

def f():
    while line[-1] != 60:
        yield np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1))
        
while line[-1] != 60:
    character, h, c = generator_model_line0.predict_generator(f(), steps=1)
    line.append(np.argmax(character))
    
line

[14, 14, 15, 16, 60]

In [110]:
generator_inputs_h = [None, Input(shape=(latent_dim,))]
generator_inputs_c = [None, Input(shape=(latent_dim,))]

generator_inputs.append(Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens)))
generator_lstms.append(LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True))

generator_lstm_out[1], generator_lstm_h[1], generator_lstm_c[1] = generator_lstms[1](
    generator_inputs[1], initial_state=[generator_inputs_h[1], generator_inputs_c[1]]
)
generator_denses.append(Dense(n_tokens, activation='softmax'))
generator_outputs.append(generator_denses[1](generator_lstm_out[1]))

generator_model_line1 = Model(
    [generator_inputs[1], generator_inputs_h[1], generator_inputs_c[1]],
    [generator_outputs[1], generator_lstm_h[1], generator_lstm_c[1]]
)
generator_model_line1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           (1, None, 61)        0                                            
__________________________________________________________________________________________________
input_33 (InputLayer)           (None, 256)          0                                            
__________________________________________________________________________________________________
input_34 (InputLayer)           (None, 256)          0                                            
__________________________________________________________________________________________________
lstm_23 (LSTM)                  [(1, None, 256), (1, 325632      input_35[0][0]                   
                                                                 input_33[0][0]                   
          

In [111]:
generator_lstms[1].set_weights(lstms[1].get_weights())
generator_denses[1].set_weights(denses[1].get_weights())

In [116]:
#generator_model_line1.reset_states()
#generator_line1_lstm.reset_states()
#generator_lstms[1].reset_states()
line = [23]
#h = np.array([np.zeros(256)])
#c = np.array([np.zeros(256)])

def f():
    while line[-1] != 60:
        yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c]
        
while line[-1] != 60:
    character, h, c = generator_model_line1.predict_generator(f(), steps=1)
    line.append(np.argmax(character))
    
line

[23, 23, 24, 25, 60]